In [112]:
#Carguemos los datos
import pandas as pd
ShanghaiDat=pd.read_csv('Shanghai-2005-2025.csv')
ShanghaiDat=ShanghaiDat.drop(ShanghaiDat.columns[0], axis=1) #Eliminar la primera columna que tiene la etiqueta

print('Dimension del dataframe:',ShanghaiDat.shape)
print('Primer elemento del dataframe:',ShanghaiDat.iloc[0,0])

Dimension del dataframe: (1, 4875)
Primer elemento del dataframe: 1242.7740478515625


In [113]:
#Nos saltamos la parte del codigo que genera datos porque nosotros ya tenemos datos reales
#Debemos convertir nuestros datos en una ventana de datos deslizantes. Esto es, obtener una matriz dividiendo los datos en epocas de 15 dias y obtener los target para cada una de esas epocas
import numpy as np

# Convertimos los datos a un array de 1 dimension
ShanghaiDat = np.array(ShanghaiDat).flatten()

#Dividimos en epocas de 15 dias
def Epocas(sequence_length=15):
    size = len(ShanghaiDat) #Es multiplo de 15
    sequences = [ShanghaiDat[i:i+sequence_length] for i in range(size-sequence_length)]
    next_points = ShanghaiDat[sequence_length:]

    # Convertir a NumPy arrays correctamente
    return np.array(sequences), np.array(next_points)

Shanghai_epocas, Shanghai_targets = Epocas()

print('Shape de Shanghai_epocas:', Shanghai_epocas.shape)
print('Shape de Shanghai_targets:', Shanghai_targets.shape)

#Ahora tenemos 4860 epocas de 15 dias en una variable, y en la otra varibale todos los targets de cada epoca. Es importante señalar que ninguna epoca puede contener a su target
#Es por ello que en la ultima epoca solo se llega al penultimo valor

Shape de Shanghai_epocas: (4860, 15)
Shape de Shanghai_targets: (4860,)


In [114]:
#Despues necesitamos definir una clase que permita tratar los datos para poder usar DataLoader de pytorch.
#Terminamos de cargar todas las librerias que vamos a usar
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

class TimeSeriesDataset(Dataset): #Heredamos caracteristicas de Dataset
    def __init__(self, sequences, next_points): #Determina que parametros debemos pasar la crear una instancia
        self.sequences = sequences
        self.next_points = next_points

    def __len__(self): #Nos dice la longitud de nuestros datos de secuancia
        return len(self.sequences)

    def __getitem__(self, idx): #Nos da cualquier epoca que queramos y su targeta asociado
        return self.sequences[idx], self.next_points[idx]

In [ ]:
#Probamos crear una instancia con nuestros datos
Datos=TimeSeriesDataset(Shanghai_epocas, Shanghai_targets)
print(len(Datos))
Datos[-1]

4860


(array([3211.39306641, 3168.52392578, 3160.75488281, 3240.93994141,
        3227.11694336, 3236.03198242, 3241.82104492, 3244.37792969,
        3242.62304688, 3213.62402344, 3230.1640625 , 3252.62597656,
        3250.60107422, 3229.48803711, 3270.65893555]),
 np.float64(3303.6669921875))

In [116]:
# Transformer Model (simplified for numerical data)
class TransformerModel(nn.Module):
    def __init__(self, input_size=1, sequence_length=10, num_layers=1, \
                 num_heads=2, dim_feedforward=512):
        super(TransformerModel, self).__init__()
        self.sequence_length = sequence_length
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=input_size*sequence_length,
                           nhead=num_heads,
                           dim_feedforward=dim_feedforward)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer,
                      num_layers=num_layers)
        self.fc_out = nn.Linear(input_size * sequence_length, 1)

    def forward(self, src):
        # Reshape to match the input dimensions
        src = src.reshape(-1, self.sequence_length, 1)  
        src = src.flatten(start_dim=1)
        src = src.unsqueeze(0)  # Add batch dimension
        out = self.transformer_encoder(src)
        out = out.squeeze(0)  # Remove batch dimension
        return self.fc_out(out)